# Redo Marks fitness pred

In [13]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import sys
from sklearn.metrics import roc_auc_score

sys.path.append("../")
from utils import *
from global_variables import *
from escape_map import *
pgm_path = "PGM/"
sys.path.append(pgm_path + "source/")
sys.path.append(pgm_path + "utilities/")
import utilities, Proteins_utils, sequence_logo, plots_utils
import rbm, RBM_utils

In [14]:

# --- Load ---
df = pd.read_csv("rbd_dist_one_scores_gisaid.csv")
df=df[df['i']>=349] 
df=df[df['i']<=526]



In [15]:
df

,Unnamed: 0,i,wt,mut,bloom_ace2_binding,bloom_expression,chan_expression,chan_ace2_binding,fitness_eve,dissimilarity_charge_hydro,accessibility_wcn,evescape,max_escape_experiment_bloom,is_escape_experiment_bloom,max_escape_experiment_xie,is_escape_experiment_xie,is_escape_experiment_all,mutation,counts,first_seen
135,135,349,S,A,-0.34,-1.13,-3.83,-2.68,-6.902246,-1.222299,-1.417140,-2.387840,0.00000,False,0.000000,False,False,S349A,53.0,NaN
136,136,349,S,C,-0.12,-2.03,-1.77,-0.78,-7.368066,-1.609191,-1.417140,-2.563646,0.00000,False,0.000000,False,False,S349C,10.0,NaN
137,137,349,S,F,-2.03,-2.50,-3.46,-2.75,-7.207227,-0.559057,-1.417140,-2.345080,0.00000,False,0.000000,False,False,S349F,16.0,NaN
138,138,349,S,G,-0.59,-2.08,-1.04,-0.77,-6.908368,-1.388110,-1.417140,-2.418569,0.00000,False,0.000000,False,False,S349G,NaN,NaN
139,139,349,S,I,-4.25,-2.69,-3.35,-1.95,-6.500989,-0.337976,-1.417140,-2.156518,0.00000,False,0.000000,False,False,S349I,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,1531,526,G,E,-0.01,-0.13,NaN,NaN,-8.738202,1.067187,-1.885148,-3.535322,0.07851,False,0.010790,False,False,G526E,NaN,NaN
1532,1532,526,G,R,-0.08,-0.30,NaN,NaN,-8.578857,3.241148,-1.885148,-3.269983,0.04760,False,0.008869,False,False,G526R,6.0,NaN
1533,1533,526,G,S,-0.02,-0.14,NaN,NaN,-8.599188,-1.388110,-1.885148,-3.868234,0.05173,False,0.008869,False,False,G526S,NaN,NaN
1534,1534,526,G,V,-0.04,-0.12,NaN,NaN,-8.664563,-1.461803,-1.885148,-3.899844,0.05142,False,0.008869,False,False,G526V,5.0,NaN


In [16]:
Y_ALL = (df["first_seen"].notna()).astype(int).to_numpy()
scores_baseline=df['evescape'].to_numpy()
#average of y
print(Y_ALL.sum()/len(Y_ALL))

0.25767309064953603


In [17]:
WT

'SVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCG'

In [18]:
def get_sequence(WT, site,	mutation):
    #site is expected to be 349 instead of 0
    site=site-349
    return WT[:site]+mutation+WT[site+1:]

#example:

df['sequence']=df.apply(lambda x: get_sequence(WT, x['i'],  x['mut']), axis=1)
    

In [19]:
with open('single_variants.fasta', 'w') as f:
    for index, row in df.iterrows():
        f.write('>' + str(index) + '\n')
        f.write(row['sequence'] + '\n')
seqs=Proteins_utils.load_FASTA('single_variants.fasta', drop_duplicates=False)

# EscapeMap scoring

In [20]:
raw_conc = np.full(len(KD_VECTORS), -8.0, dtype=np.float64)  # antibodies
model = EscapeMap(
    rbm=RBM,
    kd_vectors=KD_VECTORS,
    ace2_vector=ACE2_KD_VECTOR,
    raw_concentrations=raw_conc,
    raw_ace2=-9,      # ACE2 concentration in log10 space
    raw_beta=-1,
)
scores=-model(seqs)
#auc to compare score and y
auc = roc_auc_score(Y_ALL, scores)
print("AUC:", auc)
#params are the individual concentrations for each Ab (dont put -12 everywhere), raw_ace2, raw_beta
# i want to find best parameters to maximize predictive power=auc, but could be something else


AUC: 0.6440163541444706


In [21]:
wanted = ["S309", "LY-CoV555", "REGN10987", "REGN10933"]

# strict: fail if any key is missing
for k in wanted:
    assert k in KD_VECTORS, f"Missing key: {k}"
kd_vector_small = {k: KD_VECTORS[k] for k in wanted}


In [22]:
auc = roc_auc_score(Y_ALL, scores_baseline)
print("AUC Evescape:", auc)

AUC Evescape: 0.6318000213083315


In [23]:
from escape_map_torch import EscapeMapTorch
raw_conc = np.full(len(KD_VECTORS), -8.0, dtype=np.float64)  # antibodies
model = EscapeMapTorch(
    rbm=RBM,
    kd_vectors=KD_VECTORS,
    ace2_vector=ACE2_KD_VECTOR,
    raw_concentrations=raw_conc,
    raw_ace2=-9,      # ACE2 concentration in log10 space
    raw_beta=-1,
)
scores=-model(seqs)
print('scores shape:', scores)
#auc to compare score and y

auc = roc_auc_score(Y_ALL, scores.detach().numpy())
print("AUC:", auc)
#params are the individual concentrations for each Ab (dont put -12 everywhere), raw_ace2, raw_beta
# i want to find best parameters to maximize predictive power=auc, but could be something else


scores shape: tensor([578.0441, 574.5410, 574.5409,  ..., 576.8531, 576.6968, 572.7727],
       dtype=torch.float64, grad_fn=<NegBackward0>)
AUC: 0.6440163541444706


In [25]:
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from math import ceil
# assumes EscapeMapTorch is defined as above, and KD_VECTORS, ACE2_KD_VECTOR, RBM, WT_SEQ exist

# ----- loss: pairwise logistic (AUC surrogate) -----
def pairwise_logistic_loss(scores: torch.Tensor, y: torch.Tensor, margin: float = 0.0, max_pairs: int = 200_000):
    # scores: [B], y: [B] in {0,1}
    pos = scores[y == 1]
    neg = scores[y == 0]
    if pos.numel() == 0 or neg.numel() == 0:
        return None  # skip batch with single class
    # all pairwise diffs
    # sample pairs if too many
    num_pairs = pos.numel() * neg.numel()
    if num_pairs > max_pairs:
        k = max_pairs
        ip = torch.randint(0, pos.numel(), (k,), device=scores.device)
        ineg = torch.randint(0, neg.numel(), (k,), device=scores.device)
        diff = pos[ip] - neg[ineg]
    else:
        diff = pos[:, None] - neg[None, :]
    # loss = log(1 + exp(-(diff - margin))) = softplus(-(diff - margin))
    return F.softplus(-(diff - margin)).mean()

# ----- optional: BCE loss (logistic classification) -----
bce = torch.nn.BCEWithLogitsLoss()

# ===== data =====
# seqs: np.ndarray [N, L] int; y: np.ndarray [N] in {0,1}
# provide your arrays:
# seqs = ...
# y = ...

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float64  # to match original

model = EscapeMapTorch(
    rbm=RBM, kd_vectors=KD_VECTORS, ace2_vector=ACE2_KD_VECTOR,
    raw_concentrations=None, raw_ace2=None, raw_beta=None,
    total_beta=1.0, device=device, dtype=dtype
).to(device)

# optimize only the three parameters
opt = torch.optim.Adam(
    [model.raw_concentrations, model.raw_beta, model.raw_ace2],
    lr=2e-1, weight_decay=1e-5
)

# ===== training =====
X = torch.as_tensor(seqs, dtype=torch.long, device=device)
Y = torch.as_tensor(Y_ALL, dtype=dtype, device=device)

epochs = 20
batch_size = 256
use_pairwise = True  # set False to train with BCE

N = X.size(0)
idx_all = torch.arange(N, device=device)

for epoch in range(1, epochs + 1):
    # shuffle
    perm = idx_all[torch.randperm(N, device=device)]
    for bi in range(ceil(N / batch_size)):
        bidx = perm[bi * batch_size : (bi + 1) * batch_size]
        xb = X[bidx]
        yb = Y[bidx]

        opt.zero_grad()
        scores = -model(xb)  # higher score = more positive
        if use_pairwise:
            loss = pairwise_logistic_loss(scores, yb, margin=0.0)
            if loss is None:
                # fallback to BCE if only one class in batch
                loss = bce(scores, yb)
        else:
            loss = bce(scores, yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(
            [model.raw_concentrations, model.raw_beta, model.raw_ace2], max_norm=10.0
        )
        opt.step()

    # evaluate AUC on full set
    with torch.no_grad():
        full_scores = -model(X).detach().cpu().numpy()
    auc = roc_auc_score(Y_ALL, full_scores)
    print(f"epoch {epoch:03d} | loss {float(loss):.4f} | AUC {auc:.4f}")

# ===== final scores & AUC =====
with torch.no_grad():
    scores = -model(X).cpu().numpy()
final_auc = roc_auc_score(Y_ALL, scores)
print("Final AUC:", final_auc)

# access learned params
rc = model.raw_concentrations.detach().cpu().numpy()
rb = float(model.raw_beta.detach().cpu())
ra = float(model.raw_ace2.detach().cpu())
print("raw_concentrations shape:", rc)
print("raw_beta:", rb, "raw_ace2:", ra)


epoch 001 | loss 0.5636 | AUC 0.6824
epoch 002 | loss 0.6534 | AUC 0.6837
epoch 003 | loss 0.5437 | AUC 0.6891
epoch 004 | loss 0.5517 | AUC 0.6967
epoch 005 | loss 0.5918 | AUC 0.6985
epoch 006 | loss 0.6395 | AUC 0.7067
epoch 007 | loss 0.5420 | AUC 0.7123
epoch 008 | loss 0.4434 | AUC 0.7144
epoch 009 | loss 0.4826 | AUC 0.7150
epoch 010 | loss 0.5650 | AUC 0.7164
epoch 011 | loss 0.5654 | AUC 0.7184
epoch 012 | loss 0.5269 | AUC 0.7184
epoch 013 | loss 0.4493 | AUC 0.7191
epoch 014 | loss 0.4288 | AUC 0.7199
epoch 015 | loss 0.5404 | AUC 0.7191
epoch 016 | loss 0.4975 | AUC 0.7176
epoch 017 | loss 0.5355 | AUC 0.7194
epoch 018 | loss 0.5664 | AUC 0.7207
epoch 019 | loss 0.5463 | AUC 0.7214
epoch 020 | loss 0.5327 | AUC 0.7213
Final AUC: 0.7213496164500319
raw_concentrations shape: [ -8.03522783  -6.40414844  -8.06499124  -9.11773589  -9.14048732
  -8.45852929 -10.17755244  -8.64598962 -10.16087679  -7.19344072
  -9.01037875 -10.66729647 -10.78945085  -8.3135514  -10.92304576
 -10.5

In [27]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from math import ceil

# ===== EscapeMapTorch with learnable total_beta =====
class EscapeMapTorch(nn.Module):
    def __init__(
        self, rbm, kd_vectors, ace2_vector,
        raw_concentrations=None, raw_ace2=None, raw_beta=None,
        total_beta=1.0, device=None, dtype=torch.float64
    ):
        super().__init__()
        import math, numpy as np
        self.device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
        self.dtype = dtype
        self.kd_vectors = kd_vectors
        self.ace2_vector = {"ace2": ace2_vector}
        self.rbm = rbm
        self.ln10 = math.log(10.0)

        A = len(kd_vectors)
        rc = torch.full((A,), -10.0, dtype=dtype, device=self.device) if raw_concentrations is None \
             else torch.as_tensor(raw_concentrations, dtype=dtype, device=self.device)
        self.raw_concentrations = nn.Parameter(rc)

        ra = -10.0 if raw_ace2 is None else float(raw_ace2)
        rb =  -2 if raw_beta is None else float(raw_beta)
        self.raw_ace2 = nn.Parameter(torch.tensor(ra, dtype=dtype, device=self.device))
        self.raw_beta = nn.Parameter(torch.tensor(rb, dtype=dtype, device=self.device))

        # NEW: learnable total_beta (positive)
        self.raw_total_beta = nn.Parameter(torch.tensor(float(np.log(total_beta)), dtype=dtype, device=self.device))

    @staticmethod
    def _to_int16_c_contig(x):
        import numpy as np
        if x.dtype != np.int16:
            x = x.astype(np.int16, copy=False)
        return np.ascontiguousarray(x)

    def _get_Kd_batch_numpy(self, seqs_np, vectors, log10=True):
        import numpy as np
        try:
            out = get_Kd(seqs_np, vectors, log10=log10)
        except Exception:
            out = np.stack([get_Kd(x, vectors, log10=log10) for x in seqs_np], axis=0)
        return np.asarray(out)

    @torch.no_grad()
    def _rbm_free_energy(self, seqs_np):
        fe = self.rbm.free_energy(seqs_np)
        return torch.as_tensor(fe, dtype=self.dtype, device=self.device)

    def forward(self, s):
        import numpy as np
        s_np = np.asarray(s)
        single = (s_np.ndim == 1)
        seqs_np = s_np[None, :] if single else s_np
        seqs_np = self._to_int16_c_contig(seqs_np)

        beta = torch.exp(self.raw_beta)
        total_beta = torch.exp(self.raw_total_beta)   # positive
        ln10 = self.ln10

        kds_np = self._get_Kd_batch_numpy(seqs_np, self.kd_vectors, log10=True) * ln10
        kds = torch.as_tensor(np.squeeze(kds_np), dtype=self.dtype, device=self.device)

        kdace2_np = self._get_Kd_batch_numpy(seqs_np, self.ace2_vector, log10=True) * ln10
        kdace2_np = np.clip(np.squeeze(kdace2_np), -15.0, -5.0)
        kdace2 = torch.as_tensor(kdace2_np, dtype=self.dtype, device=self.device)

        conc = self.raw_concentrations * ln10
        def softplus_stable(x):
            out = torch.empty_like(x)
            m = x > 1
            out[m] = x[m] + torch.log1p(torch.exp(-x[m]))
            out[~m] = torch.log1p(torch.exp(x[~m]))
            return out

        energy = softplus_stable(-kds + conc).sum(dim=-1)
        energy = energy + softplus_stable(kdace2 - self.raw_ace2)

        fe = self._rbm_free_energy(seqs_np)
        energy = (energy + beta * fe) * total_beta
        return energy[0] if single else energy

# ===== Training with BCE on sigmoid(score - WT) =====
def fit_escape_map_sigmoid_diff(
    seqs, y, rbm, kd_vectors, ace2_vector,
    epochs=50, batch_size=512, lr=2e-1, weight_decay=1e-5,
    device=None, dtype=torch.float64
):
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    X = torch.as_tensor(seqs, dtype=torch.long, device=device)
    Y = torch.as_tensor(y.astype(float), dtype=dtype, device=device)

    model = EscapeMapTorch(
        rbm=rbm, kd_vectors=kd_vectors, ace2_vector=ace2_vector,
        total_beta=1.0, device=device, dtype=dtype
    ).to(device)

    # Optimizer: now include raw_concentrations
    params = [model.raw_concentrations, model.raw_beta, model.raw_ace2, model.raw_total_beta]
    opt = torch.optim.Adam(params, lr=lr, weight_decay=weight_decay)

    # Class imbalance handling
    pos = float(Y.sum().item()); neg = float(Y.numel() - pos)
    pos_weight = torch.tensor(neg / max(pos, 1.0), dtype=dtype, device=device)
    bce = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    N = X.size(0); idx_all = torch.arange(N, device=device)
    WT = torch.as_tensor(WT_SEQ, dtype=torch.long, device=device)

    for _ in range(epochs):
        perm = idx_all[torch.randperm(N, device=device)]
        for bi in range(ceil(N / batch_size)):
            bidx = perm[bi * batch_size : (bi + 1) * batch_size]
            xb, yb = X[bidx], Y[bidx]

            opt.zero_grad()
            wt_score = -model(WT)               # scalar
            scores_b = -model(xb)               # [B]
            logits = scores_b - wt_score        # classification margin vs WT
            loss = bce(logits, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(params, max_norm=10.0)
            opt.step()
            print('model.raw_beta:', model.raw_beta.item())
            print('model.raw_ace2:', model.raw_ace2.item())

    # Final evaluation
    with torch.no_grad():
        wt_score = -model(WT)
        scores = -model(X)
        logits_full = scores - wt_score
        prob = torch.sigmoid(logits_full).cpu().numpy()
        auc = roc_auc_score(y, prob)
        rc = model.raw_concentrations.detach().cpu().numpy()
        rb = float(model.raw_beta.detach().cpu())
        ra = float(model.raw_ace2.detach().cpu())
        tb = float(torch.exp(model.raw_total_beta).detach().cpu())  # total_beta in linear scale

    return {
        "model": model,
        "prob": prob,
        "AUC": auc,
        "raw_concentrations": rc,
        "raw_beta": rb,
        "raw_ace2": ra,
        "total_beta": tb,
    }


# out = fit_escape_map_sigmoid_diff(
#     seqs=seqs, y=Y_ALL, rbm=RBM, kd_vectors=KD_VECTORS, ace2_vector=ACE2_KD_VECTOR,
#     epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5,
# )
# print(out["AUC"], out["raw_ace2"], out["raw_beta"], out["total_beta"], out["raw_concentrations"])
# # --- save scores ---
# scores_df = pd.DataFrame({
#     "seq_index": np.arange(len(seqs)),
#     "y": Y_ALL,
#     "score": out["prob"]
# })
# scores_df.to_csv("scores_sigmoid_diff.csv", index=False)

# # --- save parameters ---
# ab_names = list(KD_VECTORS.keys())
# param_row = {
#     "AUC": out["AUC"],
#     "raw_beta": out["raw_beta"],
#     "raw_ace2": out["raw_ace2"],
#     "total_beta": out["total_beta"],
# }
# for name, val in zip(ab_names, out["raw_concentrations"]):
#     param_row[f"raw_c_{name}"] = float(val)

# params_df = pd.DataFrame([param_row])
# params_df.to_csv("params_sigmoid_diff.csv", index=False)




# out = fit_escape_map_sigmoid_diff(
#     seqs=seqs, y=Y_ALL, rbm=RBM, kd_vectors=kd_vector_small, ace2_vector=ACE2_KD_VECTOR,
#     epochs=30, batch_size=2048, lr=2e-1, weight_decay=1e-5,
# )
# print(out["AUC"], out["raw_ace2"], out["raw_beta"], out["total_beta"], out["raw_concentrations"])
# # --- save scores ---
# scores_df = pd.DataFrame({
#     "seq_index": np.arange(len(seqs)),
#     "y": Y_ALL,
#     "score": out["prob"]
# })
# scores_df.to_csv("scores_sigmoid_diff_4ab.csv", index=False)

# # --- save parameters ---
# ab_names = list(kd_vector_small.keys())
# param_row = {
#     "AUC": out["AUC"],
#     "raw_beta": out["raw_beta"],
#     "raw_ace2": out["raw_ace2"],
#     "total_beta": out["total_beta"],
# }
# for name, val in zip(ab_names, out["raw_concentrations"]):
#     param_row[f"raw_c_{name}"] = float(val)

# params_df = pd.DataFrame([param_row])
# params_df.to_csv("params_sigmoid_diff_4ab.csv", index=False)

In [ ]:
import numpy as np
import torch
from math import ceil
from sklearn.metrics import roc_auc_score

# --- stratified subset indices ---
def stratified_subset_indices(y, frac, seed=0):
    y = np.asarray(y).astype(int)
    rng = np.random.default_rng(seed)
    pos = rng.choice(np.flatnonzero(y==1), size=max(1, int(np.ceil(frac*(y==1).sum()))), replace=False)
    neg = rng.choice(np.flatnonzero(y==0), size=max(1, int(np.ceil(frac*(y==0).sum()))), replace=False)
    idx = np.concatenate([pos, neg]); rng.shuffle(idx); return idx

# --- unchanged model class: EscapeMapTorch ---

def fit_escape_map_sigmoid_diff(
    seqs, y, rbm, kd_vectors, ace2_vector,
    epochs=50, batch_size=512, lr=2e-1, weight_decay=1e-5,
    device=None, dtype=torch.float64, train_idx=None, seed=0
):
    torch.manual_seed(seed); np.random.seed(seed)
    device = device or (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
    X = torch.as_tensor(seqs, dtype=torch.long, device=device)
    Y = torch.as_tensor(np.asarray(y).astype(float), dtype=dtype, device=device)

    # subset for training
    if train_idx is None:
        Xtr, Ytr = X, Y
    else:
        ti = torch.as_tensor(train_idx, dtype=torch.long, device=device)
        Xtr, Ytr = X[ti], Y[ti]

    model = EscapeMapTorch(rbm, kd_vectors, ace2_vector, total_beta=1.0, device=device, dtype=dtype).to(device)
    params = [model.raw_concentrations, model.raw_beta, model.raw_ace2, model.raw_total_beta]
    opt = torch.optim.Adam(params, lr=lr, weight_decay=weight_decay)

    # class weight from the subset
    pos = float(Ytr.sum().item()); neg = float(Ytr.numel() - pos)
    pos_weight = torch.tensor(neg / max(pos, 1.0), dtype=dtype, device=device)
    bce = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    WT = torch.as_tensor(WT_SEQ, dtype=torch.long, device=device)
    Ntr = Xtr.size(0); idx_local = torch.arange(Ntr, device=device)

    for _ in range(epochs):
        perm = idx_local[torch.randperm(Ntr, device=device)]
        for bi in range(ceil(Ntr / batch_size)):
            bidx = perm[bi*batch_size : (bi+1)*batch_size]
            xb, yb = Xtr[bidx], Ytr[bidx]
            opt.zero_grad()
            logits = (-model(xb)) - (-model(WT))   # margin vs WT
            loss = bce(logits, yb)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(params, 10.0)
            opt.step()

    with torch.no_grad():
        logits_full = (-model(X)) - (-model(WT))
        prob = torch.sigmoid(logits_full).cpu().numpy()
        auc = roc_auc_score(np.asarray(y).astype(int), prob)

    return {"model": model, "prob": prob, "AUC": auc,
            "raw_concentrations": model.raw_concentrations.detach().cpu().numpy(),
            "raw_beta": float(model.raw_beta.detach().cpu()),
            "raw_ace2": float(model.raw_ace2.detach().cpu()),
            "total_beta": float(torch.exp(model.raw_total_beta).detach().cpu())}

# --- few-shot runs ---
import pandas as pd

fracs = [0.10, 0.20, 0.30, 0.40, 0.50]

def run_fewshot(kd_vecs, tag, fracs=fracs, seed=42):
    for f in fracs:
        idx = stratified_subset_indices(Y_ALL, frac=f, seed=seed)
        out = fit_escape_map_sigmoid_diff(
            seqs=seqs, y=Y_ALL, rbm=RBM, kd_vectors=kd_vecs, ace2_vector=ACE2_KD_VECTOR,
            epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5, train_idx=idx, seed=seed
        )
        print(f"[{tag}] frac={f:.2f} AUC={out['AUC']:.4f}")

        # --- save scores (evaluate on all samples) ---
        scores_df = pd.DataFrame({
            "seq_index": np.arange(len(seqs)),
            "y": np.asarray(Y_ALL).astype(int),
            "score": out["prob"],
            "train_frac": f,
            "ab_set": tag
        })
        scores_df.to_csv(f"scores_sigmoid_diff_{tag}_{int(f*100)}.csv", index=False)

        # --- save parameters ---
        ab_names = list(kd_vecs.keys())
        param_row = {
            "AUC": out["AUC"],
            "raw_beta": out["raw_beta"],
            "raw_ace2": out["raw_ace2"],
            "total_beta": out["total_beta"],
            "train_frac": f,
            "ab_set": tag,
            "n_train": out.get("n_train", len(idx)),
            "pos_train": out.get("pos_train", int(np.asarray(Y_ALL)[idx].sum()))
        }
        for name, val in zip(ab_names, out["raw_concentrations"]):
            param_row[f"raw_c_{name}"] = float(val)

        params_df = pd.DataFrame([param_row])
        params_df.to_csv(f"params_sigmoid_diff_{tag}_{int(f*100)}.csv", index=False)

# run on both antibody sets if desired
# run_fewshot(KD_VECTORS, "allab")
# run_fewshot(kd_vector_small, "4ab")



[allab] frac=0.10 AUC=0.6449
[allab] frac=0.20 AUC=0.6620
[allab] frac=0.30 AUC=0.6750
[allab] frac=0.40 AUC=0.6463
[allab] frac=0.50 AUC=0.6617
[4ab] frac=0.10 AUC=0.7110
[4ab] frac=0.20 AUC=0.7223
[4ab] frac=0.30 AUC=0.7004
[4ab] frac=0.40 AUC=0.6998
[4ab] frac=0.50 AUC=0.7236


# Stability of parameters

In [33]:
import numpy as np
import pandas as pd

# 1) parse the date column
col = "first seen" if "first seen" in df.columns else "first_seen"
df["first_seen_dt"] = pd.to_datetime(df[col], format="%Y-%m", errors="coerce")

# 2) 6-month periods starting 2021-01-01
start = pd.Timestamp("2020-01-01")
end = df["first_seen_dt"].max(skipna=True)
print("Date range:", start, "to", end)
check_starts = pd.date_range(start, end, freq="12MS")              # 2021-01, 2021-07, ...
check_ends = check_starts + pd.DateOffset(months=12)

# 3) 1 if first_seen is within the 6-month window [start, end)
dates = df["first_seen_dt"].values[:, None]                       # (N,1) datetime64[ns]
starts = check_starts.values[None, :]                             # (1,M)
ends = check_ends.values[None, :]                                 # (1,M)
valid = (~pd.isna(df["first_seen_dt"]).values)[:, None]           # (N,1) bool
y_mat = (valid & (dates >= starts) & (dates < ends)).astype(int)  # (N,M)
# y_mat = (valid  & (dates < ends)).astype(int)  # (N,M)

# 4) attach as columns: y_YYYY-MM (period start)
y_df = pd.DataFrame(
    y_mat,
    index=df.index,
    columns=[f"y_{d.strftime('%Y-%m')}" for d in check_starts]
)
print("Last date of first_seen:", df["first_seen_dt"].max())
df_out = pd.concat([df, y_df], axis=1)


Date range: 2020-01-01 00:00:00 to 2022-08-01 00:00:00
Last date of first_seen: 2022-08-01 00:00:00


In [34]:
import os
import numpy as np
import pandas as pd
from math import ceil
from sklearn.metrics import roc_auc_score

# fit_escape_map_sigmoid_diff unchanged except that "y" is the period vector you pass in

def train_all_periods_and_save(
    df_out, seqs, rbm, kd_vecs, ace2_vec,
    tag, epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5,
    out_prefix="sigmoid_diff"
):
    period_cols = sorted([c for c in df_out.columns if c.startswith("y_")])
    ab_names = list(kd_vecs.keys())
    os.makedirs("results_periods", exist_ok=True)

    for col in period_cols:
        y = df_out[col].to_numpy().astype(int)
        # skip degenerate periods
        if y.sum() == 0 or y.sum() == y.size:
            print(f"skip {col}: single class ({y.sum()}/{y.size})")
            continue

        out = fit_escape_map_sigmoid_diff(
            seqs=seqs, y=y, rbm=rbm, kd_vectors=kd_vecs, ace2_vector=ace2_vec,
            epochs=epochs, batch_size=batch_size, lr=lr, weight_decay=weight_decay
        )

        # AUC may fail if labels collapse numerically; guard already inside fit. If needed:
        try:
            auc = roc_auc_score(y, out["prob"])
        except Exception:
            auc = np.nan

        # ----- save scores -----
        scores_df = pd.DataFrame({
            "seq_index": np.arange(len(seqs)),
            "y": y,
            "score": out["prob"],
            "period": col.replace("y_", ""),   # YYYY-MM
            "ab_set": tag
        })
        scores_path = f"results_periods/scores_{out_prefix}_{tag}_{col}.csv"
        scores_df.to_csv(scores_path, index=False)

        # ----- save parameters -----
        param_row = {
            "period": col.replace("y_", ""),
            "ab_set": tag,
            "AUC": float(auc),
            "raw_beta": float(out["raw_beta"]),
            "raw_ace2": float(out["raw_ace2"]),
            "total_beta": float(out["total_beta"]),
        }
        for name, val in zip(ab_names, out["raw_concentrations"]):
            param_row[f"raw_c_{name}"] = float(val)

        params_df = pd.DataFrame([param_row])
        params_path = f"results_periods/params_{out_prefix}_{tag}_{col}.csv"
        params_df.to_csv(params_path, index=False)

        print(f"{tag} {col}: AUC={auc:.4f} -> {scores_path}, {params_path}")

# ---- run for each antibody set ----
train_all_periods_and_save(
    df_out=df_out, seqs=seqs, rbm=RBM, kd_vecs=KD_VECTORS, ace2_vec=ACE2_KD_VECTOR, tag="allab",
    epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5
)
train_all_periods_and_save(
    df_out=df_out, seqs=seqs, rbm=RBM, kd_vecs=kd_vector_small, ace2_vec=ACE2_KD_VECTOR, tag="4ab",
    epochs=30, batch_size=1024, lr=2e-1, weight_decay=1e-5
)


allab y_2020-01: AUC=0.6496 -> results_periods/scores_sigmoid_diff_allab_y_2020-01.csv, results_periods/params_sigmoid_diff_allab_y_2020-01.csv
allab y_2021-01: AUC=0.7016 -> results_periods/scores_sigmoid_diff_allab_y_2021-01.csv, results_periods/params_sigmoid_diff_allab_y_2021-01.csv
allab y_2022-01: AUC=0.6424 -> results_periods/scores_sigmoid_diff_allab_y_2022-01.csv, results_periods/params_sigmoid_diff_allab_y_2022-01.csv
4ab y_2020-01: AUC=0.6635 -> results_periods/scores_sigmoid_diff_4ab_y_2020-01.csv, results_periods/params_sigmoid_diff_4ab_y_2020-01.csv
4ab y_2021-01: AUC=0.7191 -> results_periods/scores_sigmoid_diff_4ab_y_2021-01.csv, results_periods/params_sigmoid_diff_4ab_y_2021-01.csv
4ab y_2022-01: AUC=0.6156 -> results_periods/scores_sigmoid_diff_4ab_y_2022-01.csv, results_periods/params_sigmoid_diff_4ab_y_2022-01.csv
